In [ ]:

# Importing all the required libaries for visualization and EDA process 
import pandas as pd
import numpy as np



In [ ]:

# Reading train & test data
# test_data = pd.read_csv('/content/drive/MyDrive/PHD Exam/test_data.csv')
train_data = pd.read_csv('/content/drive/MyDrive/PHD Exam/train_data.csv')



In [ ]:

# Reading other data also
foot_fall = pd.read_csv("/content/drive/MyDrive/PHD Exam/foot_fall.csv")
dis_freature = pd.read_csv("/content/drive/MyDrive/PHD Exam/discount_features.csv")



In [ ]:

import json



In [ ]:

with open('/content/drive/MyDrive/PHD Exam/city_dict.json') as f:
  city_dict = json.load(f)




<h1 id="Data-Manipulation-and-creating-base-data-after-merging-other-data-file">Data Manipulation and creating base data after merging other data file<a class="anchor-link" href="#Data-Manipulation-and-creating-base-data-after-merging-other-data-file">¶</a></h1>



<ul>
<li>Quick check all the data</li>
</ul>


In [ ]:

# Train data
train_data.head()



In [ ]:

df_sample= train_data.groupby(['year','month','day','city','medicine']).apply(lambda x: x.sample(frac=0.20))
df_sample.shape



In [ ]:

# Test data
test_data.head()



In [ ]:

# foot fall data
foot_fall.head()



In [ ]:

# discount feature
dis_freature.head()



In [ ]:

# City Dictonery
city_dict = pd.DataFrame.from_dict(city_dict,orient='index')
city_dict



In [ ]:

city_dict = city_dict.reset_index()
city_dict = city_dict.rename(columns={city_dict.columns[0]:'city',city_dict.columns[1]:'city_name'})
city_dict



In [ ]:

# Changing city date-type so i can mapped wiht base data
city_dict['city'] = city_dict["city"].astype(int)




<p><em>Creating based data joining all required data</em></p>


In [ ]:

# Creating date columns with year,month,day
train_data["date"] = pd.to_datetime(train_data[['year', 'month', 'day']])
train_data.head()



In [ ]:

df_sample= train_data.groupby(['date','city']).apply(lambda x: x.sample(frac=0.20))
df_sample.shape



In [ ]:

df_sample = df_sample.reset_index(drop=True)
df_sample.head()



In [ ]:

df_sample.to_csv("/content/drive/MyDrive/PHD Exam/df_sample.csv")




<h4 id="Creating-final-data-after-mapping-all-data">Creating final data after mapping all data<a class="anchor-link" href="#Creating-final-data-after-mapping-all-data">¶</a></h4>


In [ ]:

# Mapping city from city_dict
final_train_df = pd.merge(train_data,city_dict,left_on="city",right_on="city",how="left")
final_train_df.head()



In [ ]:

# Melting footfall data so we can mapped with final_train_df data
idcol = "city"
varcol = np.setdiff1d(foot_fall.columns,idcol)
foot_fall_melted = pd.melt(foot_fall,id_vars=idcol,value_vars=varcol)
foot_fall_melted["variable"] = foot_fall_melted["variable"].astype('datetime64[ns]')
foot_fall_melted = foot_fall_melted.rename(columns={'value':'footfall_count'})
foot_fall_melted.head()



In [ ]:

# Mapping with base data - "final_train_df" based of city and variable
final_train_df = pd.merge(final_train_df,foot_fall_melted,left_on=["date",'city_name'],right_on=['variable',"city"],how="left")
final_train_df.head()



In [ ]:

# Removing duplicte colums
final_train_df = final_train_df.drop(["variable",'city_y'],axis = 1)
final_train_df = final_train_df.rename(columns={'city_x':'city'})
final_train_df.head()



In [ ]:

# Shape of data frame after mergging all nessary data
final_train_df.shape



In [ ]:

# Droping all duplicate rows if it exit in dateframe
final_train_df = final_train_df.drop_duplicates()
final_train_df.shape




<p><em>Key Finding</em></p>
<p>1.Base data contain duplicate rows  so removing and processing further</p>



<h2 id="Descriptive-Statitics">Descriptive Statitics<a class="anchor-link" href="#Descriptive-Statitics">¶</a></h2>


In [ ]:

# Checking basic information of dataframe
final_train_df.info()



In [ ]:

# Univariate analysis of data which give some basic understanding of each and every variable in dataframe
final_train_df.describe()




<p><em>Key Finding and solution</em></p>
<ol>
<li>As we can see above sales contain negative value.</li>
</ol>
<p><em>Solution</em></p>
<ol>
<li>We can convert negative sales value as zero(0) sales value. </li>
<li>We can remove those rows which contain -ve sales value.</li>
</ol>


In [ ]:

#missing data
total = final_train_df.isnull().sum().sort_values(ascending=False)
percent = (final_train_df.isnull().sum()/final_train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data




<p><em>Key Finding and solution</em></p>
<ol>
<li>Footfall_count contain .25% value of missing value.</li>
</ol>
<p><em>Solution</em></p>
<ol>
<li>It is very small portion of total values so we can impute missing value or we can remove rows which contain missing values.</li>
</ol>


In [ ]:

# unqiue value for few columns of data
print("unique value in year are :",final_train_df["year"].unique())
print('\n')
print("unique value in month are :",final_train_df["month"].unique())
print('\n')
print("unique value in city_name :",final_train_df["city_name"].unique())
print('\n')
print("unique value in medicine are :",final_train_df["medicine"].unique())
print('\n')
print("unique value in day are :",final_train_df["day"].unique())




<h1 id="Data-Visualization">Data Visualization<a class="anchor-link" href="#Data-Visualization">¶</a></h1>


In [ ]:

# Importing all required libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls



In [ ]:

# To pass the warning messages
import warnings
warnings.filterwarnings('ignore')




<p><strong>Note</strong></p>
<h2 id="Data-is-quite-huge-not-able-to-do-visualization-EDA,-Below-are-few-sample-of-visualization-and-google-colab-is-crashing-very-frequently-and-picking-ranmdom-sample-from-data-and-performing-visualization-EDA">Data is quite huge not able to do visualization EDA, Below are few sample of visualization and google colab is crashing very frequently and picking ranmdom sample from data and performing visualization EDA<a class="anchor-link" href="#Data-is-quite-huge-not-able-to-do-visualization-EDA,-Below-are-few-sample-of-visualization-and-google-colab-is-crashing-very-frequently-and-picking-ranmdom-sample-from-data-and-performing-visualization-EDA">¶</a></h2>


In [ ]:

# Creating two seprate list of categorical and numerical variable 
cat_variable = ['year', 'month','day','city','city_name']
num_variable = ['sales','footfall_count','medicine']



In [ ]:

# Box plot between Year and sales 
sns.boxplot(data = final_train_df, x='year', y='sales')



In [ ]:

sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})
sns.distplot(
    final_train_df['sales'], bins=20, hist_kws={"alpha": 1}
).set(xlabel='year', ylabel='sales');



In [ ]:

# Box plot for categorical variable
fig, ax = plt.subplots(3, 3, figsize=(15, 10))
for var, subplot in zip(cat_variable, ax.flatten()):
    sns.boxplot(x=var, y='sales', data=final_train_df, ax=subplot)




<h3 id="Random-sampling-from-train-data-and-performing-EDA-process">Random sampling from train data and performing EDA process<a class="anchor-link" href="#Random-sampling-from-train-data-and-performing-EDA-process">¶</a></h3>


In [ ]:

final_train_df.shape



In [ ]:

# Randomly selecting 25% of the base data = "final_train_df" 
train_data_random_data = final_train_df.sample(frac=0.25)
train_data_random_data.shape



In [ ]:

train_data_random_data.head()



In [ ]:

# scatterplot between year and sales 
sns.scatterplot(x=train_data_random_data['year'], y=train_data_random_data['sales'])



In [ ]:

# scatterplot between month and sales 
sns.scatterplot(x=train_data_random_data['month'], y=train_data_random_data['sales'])



In [ ]:

# scatterplot between city_name and sales
k = sns.scatterplot(x=train_data_random_data['city_name'],y=train_data_random_data['sales'])
plt.xticks(rotation=45)



In [ ]:

# scatterplot between day and sales
sns.scatterplot(x=train_data_random_data['day'],y=train_data_random_data['sales'])
plt.xticks(rotation=45)




<p><em>Key Finding</em></p>
<ol>
<li>Year - as we can in scatterplot between sales and year there are some outlier in 2018.</li>
<li>Month - scatterplot between sales and month there are some outlier
between 4 and 12 months.</li>
<li>Days - scatterplot between sales and day there are some outlier
between 15 -20 days.</li>
<li>city_name - outlier follow same trend in city as we can see delhi has maximum outliear available .</li>
</ol>



<h1 id="Working-with-base-model">Working with base model<a class="anchor-link" href="#Working-with-base-model">¶</a></h1>


In [ ]:

# Importing all required libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls



In [ ]:

# Creating date columns with year,month,day
train_data["date"] = pd.to_datetime(train_data[['year', 'month', 'day']])
train_data.head()



In [ ]:

train_data.shape



In [ ]:

train_data = train_data[['date','city','medicine','sales']]
train_data.head()



In [ ]:

df_sample= train_data.groupby(['date','city']).apply(lambda x: x.sample(frac=0.20))
df_sample.shape



In [ ]:

df_sample = df_sample.reset_index(drop = True)
df_sample.head()



In [ ]:

df_sample = df_sample[["date","sales"]].reset_index(drop=True)
df_sample.head()



In [ ]:

df_sample.shape



In [ ]:

# assigning date as a index and again group by date
df_sample= df_sample.groupby(['date'])['sales'].sum()
df_sample.shape



In [ ]:

df_sample.plot()



In [ ]:

sales_mean = df_sample.rolling(window=90).mean()



In [ ]:

sales_mean.plot()




<h4 id="As-we-can-see-there-is-bit-of-trend-in-data">As we can see there is bit of trend in data<a class="anchor-link" href="#As-we-can-see-there-is-bit-of-trend-in-data">¶</a></h4>



<h1 id="Base-Line-model---Moving-avg">Base Line model - Moving avg<a class="anchor-link" href="#Base-Line-model---Moving-avg">¶</a></h1>


In [ ]:

sales_df = pd.concat([df_sample,df_sample.shift(1)],axis=1)
sales_df = sales_df.reset_index(drop = True)
sales_df.head()



In [ ]:

sales_df.columns = ["Actual_sales",'forecast_sales']
sales_df.head()



In [ ]:

# Importing model library
from sklearn.metrics import mean_squared_error



In [ ]:

# calculating error
sales_error = mean_squared_error(sales_df["Actual_sales"].iloc[1:],sales_df["forecast_sales"].iloc[1:])
sales_error



In [ ]:

np.sqrt(sales_error)



In [ ]:

df_sample.describe()




<h4 id="It-is-naive-model,-which-describe-that-we-have-a-error-of-+--59302.50-which-is-near-or-std-deviation-and-advance-models-errro-should-be-less-then-this.-It-is-very-basic-base-line-model">It is naive model, which describe that we have a error of +- 59302.50 which is near or std deviation and advance models errro should be less then this. It is very basic base line model<a class="anchor-link" href="#It-is-naive-model,-which-describe-that-we-have-a-error-of-+--59302.50-which-is-near-or-std-deviation-and-advance-models-errro-should-be-less-then-this.-It-is-very-basic-base-line-model">¶</a></h4>



<h1 id="ARIMA-=-Autoregressive-(p)-Integrated-(d)-Moving-Avg-(q)">ARIMA = Autoregressive (p) Integrated (d) Moving Avg (q)<a class="anchor-link" href="#ARIMA-=-Autoregressive-(p)-Integrated-(d)-Moving-Avg-(q)">¶</a></h1><h5 id="Moving-Avg-=-Smooting-part(It-include-how-to-remove-noise-and-error-part-from-data)">Moving Avg = Smooting part(It include how to remove noise and error part from data)<a class="anchor-link" href="#Moving-Avg-=-Smooting-part(It-include-how-to-remove-noise-and-error-part-from-data)">¶</a></h5><h5 id="Autoregressive-=-How-current-value-is-co-related-with-previous-value-and-previous-value-is-the-best-reflexion-of-the-current-value">Autoregressive = How current value is co-related with previous value and previous value is the best reflexion of the current value<a class="anchor-link" href="#Autoregressive-=-How-current-value-is-co-related-with-previous-value-and-previous-value-is-the-best-reflexion-of-the-current-value">¶</a></h5><h5 id="Integrated-=-Order-of-difference-(Series-should-be-stationary)">Integrated = Order of difference (Series should be stationary)<a class="anchor-link" href="#Integrated-=-Order-of-difference-(Series-should-be-stationary)">¶</a></h5><h6 id="For-p-and-q-we-have-ACF(Auto-corealation)-curve-and-PACF(Partial-auto-corealtion)-curve.">For p and q we have ACF(Auto corealation) curve and PACF(Partial auto corealtion) curve.<a class="anchor-link" href="#For-p-and-q-we-have-ACF(Auto-corealation)-curve-and-PACF(Partial-auto-corealtion)-curve.">¶</a></h6>


In [ ]:

# for ACF and PACF
import statsmodels
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf  



In [ ]:

# Ploting plot_acf is to identify parameter Q
# ARIMA require 3 parameter = p,d,q
plot_acf(df_sample)




<h1 id="Building-another-model">Building another model<a class="anchor-link" href="#Building-another-model">¶</a></h1>


In [ ]:

train_data.head()



In [ ]:

#Gain groping by city and medicine and sum by sales
dataset= train_data.pivot_table(index=["city","medicine"],values='sales',columns=["date"],aggfunc="sum")
dataset.shape



In [ ]:

# picking same sample from data
dataset=dataset.iloc[:,1000:1269]
dataset.head()



In [ ]:

# reseting index of data
dataset.reset_index(inplace=True)
dataset



In [ ]:

# Reading test data
test_data = pd.read_csv('/content/drive/MyDrive/PHD Exam/test_data.csv')
# Mapping test data with sample data
dataset=pd.merge(test_data,dataset,on=["medicine","city"],how="left")
# Droping unessary columns from data
dataset.drop(['id','year','month','day','medicine','city'],inplace = True, axis = 1)



In [ ]:

cols =dataset.columns
cols



In [ ]:

dataset = dataset.iloc[:5000]
dataset



In [ ]:

# Filing na value with lower value from respective row in dataframe 
for i in cols:
  dataset.astype("float16")
  dataset.fillna('bfill',inplace=True)
return dataset.head()



In [ ]:

import gc
gc.collect()




<h3 id="Starting-model-building-process">Starting model building process<a class="anchor-link" href="#Starting-model-building-process">¶</a></h3>


In [ ]:

# X we will keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)



In [ ]:

# importing libraries required for our model
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



In [ ]:

# # our defining our model 
model_lstm = Sequential()
model_lstm.add(LSTM(units = 64,input_shape = (X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dropout(0.4))
model_lstm.add(Dense(1))

model_lstm.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
model_lstm.summary()



In [ ]:

#history_lstm = model_lstm.fit(X_train,y_train,batch_size = 4096,epochs = 10)



In [ ]:

# # Plot the loss curves for training
import matplotlib.pyplot as plt
plt.plot(history_lstm.history['loss'], color='b', label="Training loss")
plt.legend(loc='best', shadow=True)



In [ ]:

# MLP for Time Series Forecasting (Multilayer Perceptron )
adam = optimizers.Adam()

model_mlp = Sequential()
model_mlp.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model_mlp.add(Dropout(0.4))
model_mlp.add(Dense(1))

model_mlp.compile(loss='mse', optimizer=adam, metrics = ['mean_squared_error'])
model_mlp.summary()



In [ ]:

history_mlp = model_mlp.fit(X_train,y_train,batch_size = 4096,epochs = 10)



In [ ]:

plt.plot(history_mlp.history['loss'], color='b', label="Training loss")
plt.legend(loc='best', shadow=True)



In [ ]:

# #CNN for Time Series Forecasting
adam = optimizers.Adam()
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss='mse', optimizer=adam)
model_cnn.summary()



In [ ]:

cnn_history = model_cnn.fit(X_train, y_train, epochs=10, verbose=2)



In [ ]:

import matplotlib.pyplot as plt
plt.plot(cnn_history.history['loss'], color='b', label="Training loss")
plt.legend(loc='best', shadow=True)



In [ ]:

#CNN-LSTM for Time Series Forecasting

#Reshape from [samples, timesteps, features] into [samples, subsequences, timesteps, features]

subsequences = 3
timesteps = X_train.shape[1]//subsequences
X_train_series_sub = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
print('Train set shape', X_train_series_sub.shape)



In [ ]:

adam = optimizers.Adam()
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(64, activation='relu',return_sequences=True))
model_cnn_lstm.add(Dropout(0.4))
model_cnn_lstm.add(LSTM(64, activation='relu',return_sequences=True))
model_cnn_lstm.add(Dropout(0.4))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer=adam)
model_cnn_lstm.summary()



In [ ]:

cnn_lstm_history = model_cnn_lstm.fit(X_train_series_sub, y_train, epochs=20, verbose=2)



In [ ]:

import matplotlib.pyplot as plt
plt.plot(cnn_lstm_history.history['loss'], color='b', label="Training loss")
plt.legend(loc='best', shadow=True)



In [ ]:

# creating submission file 
submission_pfs = model_cnn_lstm.predict(X_test)
we will keep every value between 0 and 20
submission_pfs = submission_pfs.clip(0,20)
# creating dataframe with required columns 
submission = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_pfs.ravel()})
# creating csv file from dataframe
submission.to_csv('sub_pfs.csv',index = False)



In [ ]:

#Reshape from [samples, timesteps, features] into [samples, subsequences, timesteps, features
subsequences = 3
timesteps = X_test.shape[1]//subsequences
X_test_series_sub = X_test.reshape((X_test.shape[0], subsequences, timesteps, 1))
print('Train set shape', X_test_series_sub.shape)



In [ ]:

submission = pd.DataFrame({'ID':test['id'],'sales':submission_pfs.ravel()})



In [ ]:

submission.to_csv('sample_submission.csv',index = False)

